In [2]:
import pandas as pd
import numpy as np

import string  
import nltk
import re
import scipy
import matplotlib.pyplot as plt
import urllib 
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error

!pip install catboost 
from catboost import CatBoostRegressor

train = pd.read_csv('train.csv')
display(train)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [3]:
train['keyword'].fillna('no keyword', inplace = True) 
train['keyword'] = train['keyword'].str.replace('%20', ' ')
train['location'].fillna('no location', inplace = True)

train['length'] = train['text'].str.len()

train.head()

,id,keyword,location,text,target,length
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88


In [4]:
#limpiando el texto

import nltk
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

#FUNCIONES UTILES

def only_letters(tweet):
    tweet = re.sub(r'http\S*', '', tweet)
    tweet = re.sub(r'[^a-z\s]', '', tweet)
    return tweet

stop_words=set(stopwords.words("english"))

def filter_stopwords(tokenized_text):
    filtered_words=[]
    for w in tokenized_text:
        if w not in stop_words:
            filtered_words.append(w)
    return filtered_words

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    lemmatized_words = []
    for word in tweet:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

def transform_to_text(tweet_words):
    return " ".join(tweet_words)

#Cleaning text

train['clean_text'] = train['text'].str.lower()

train['clean_text'] = train['clean_text'].apply(only_letters)    

#Tokenización

train['clean_text'] = train['clean_text'].apply(word_tokenize)

#Remove stopwords

train['clean_text'] = train['clean_text'].apply(filter_stopwords) 

#Lemmatization                                                                       

train['clean_text'] = train['clean_text'].apply(lemmatize_tweet)  

train['clean_text'] = train['clean_text'].apply(transform_to_text)
display(train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,keyword,location,text,target,length,clean_text
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69,deed reason earthquake may allah forgive u
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38,forest fire near la ronge sask canada
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133,resident asked shelter place notified officer ...
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65,people receive wildfire evacuation order calif...
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88,got sent photo ruby alaska smoke wildfire pour...
...,...,...,...,...,...,...,...
7608,10869,no keyword,no location,Two giant cranes holding a bridge collapse int...,1,83,two giant crane holding bridge collapse nearby...
7609,10870,no keyword,no location,@aria_ahrary @TheTawniest The out of control w...,1,125,ariaahrary thetawniest control wild fire calif...
7610,10871,no keyword,no location,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,65,utckm volcano hawaii
7611,10872,no keyword,no location,Police investigating after an e-bike collided ...,1,137,police investigating ebike collided car little...


In [5]:
#agregando algunos features

keywords_list = list(set(train.keyword))  #todas las keywords únicas

locations_list = list(set(train.location)) #todas las locaciones únicas
locations_list.remove('no location')

def amount_keywords(tweet):
    cont = 0
    for keyword in keywords_list:
        if keyword in tweet:
            cont += 1
    return cont

train['amount_words'] = train['clean_text'].str.split(' ').apply(lambda x: len(x))  #conviene que cuente del limpio o del original??

train['amount_keywords_in_tweet'] = train['clean_text'].apply(lambda x: amount_keywords(x))


tiene_mas_de_un_kw = train[train['amount_keywords_in_tweet'] > 1]

tiene_un_solo_kw = train[train['amount_keywords_in_tweet'] == 1]

no_tiene_kw = train[train['amount_keywords_in_tweet'] == 0]

porcentaje_target_mas_de_un_kw = tiene_mas_de_un_kw['target'].mean()
print("porcentaje target de tweets con mas de un kw", porcentaje_target_mas_de_un_kw)

porcentaje_target_un_solo_kw = tiene_un_solo_kw['target'].mean()
print("porcentaje target de tweets con un solo kw", porcentaje_target_un_solo_kw)

porcentaje_target_sin_kw = no_tiene_kw['target'].mean()
print("porcentaje target de tweets sin kw", porcentaje_target_sin_kw)

print("el maximo de cant de keywords en un tweet es", tiene_mas_de_un_kw['amount_keywords_in_tweet'].max())

def amount_locations(tweet):    #esta función podría unificarse con la de keywords pasandole por parámetro la lista a recorrer
    cont = 0
    for location in locations_list:
        if location in tweet:
            cont += 1
    return cont

train['amount_locations_in_tweet'] = train['clean_text'].apply(lambda x: amount_locations(x))

mas_de_1_location = train[train['amount_locations_in_tweet'] > 1]

un_solo_location = train[train['amount_locations_in_tweet'] == 1]

no_tiene_location = train[train['amount_locations_in_tweet'] == 0]

porcentaje_target_mas_de_un_location = mas_de_1_location['target'].mean()
print("porcentaje target de tweets con mas de una locacion", porcentaje_target_mas_de_un_location)

porcentaje_target_un_solo_location = un_solo_location['target'].mean()
print("porcentaje target de tweets con una sola locacion", porcentaje_target_un_solo_location)

porcentaje_target_sin_location = no_tiene_location['target'].mean()
print("porcentaje target de tweets sin locacion", porcentaje_target_sin_location)

print("el maximo de cant de locaciones en un tweet es", mas_de_1_location['amount_locations_in_tweet'].max())

porcentaje target de tweets con mas de un kw 0.606816584680253
porcentaje target de tweets con un solo kw 0.3446327683615819
porcentaje target de tweets sin kw 0.2025862068965517
el maximo de cant de keywords en un tweet es 7
porcentaje target de tweets con mas de una locacion 0.504933504933505
porcentaje target de tweets con una sola locacion 0.4489409984871407
porcentaje target de tweets sin locacion 0.34382107657316147
el maximo de cant de locaciones en un tweet es 8


In [6]:
mas_de_un_location_y_mas_de_un_keyword = train[(train['amount_locations_in_tweet'] > 1) & (train['amount_keywords_in_tweet'] > 1)]
porcentaje_target_mas_de_un_location_y_mas_de_un_keyword = mas_de_un_location_y_mas_de_un_keyword.target.mean()
porcentaje_target_mas_de_un_location_y_mas_de_un_keyword

0.6990196078431372

In [8]:
#construyendo los datos de entrenamiento y de test
selected_df = train[['length','amount_words','amount_keywords_in_tweet','amount_locations_in_tweet']] 
#ver después de codificar las variables categóricas para agregarlas
X, y = selected_df,train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [9]:
#Catboost
model = CatBoostRegressor()
model.fit(X_train, y_train)
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

Learning rate set to 0.053835
0:	learn: 0.4919208	total: 180ms	remaining: 3m
1:	learn: 0.4889263	total: 183ms	remaining: 1m 31s
2:	learn: 0.4862268	total: 188ms	remaining: 1m 2s
3:	learn: 0.4840966	total: 191ms	remaining: 47.5s
4:	learn: 0.4817821	total: 194ms	remaining: 38.6s
5:	learn: 0.4795994	total: 199ms	remaining: 32.9s
6:	learn: 0.4776192	total: 202ms	remaining: 28.6s
7:	learn: 0.4759245	total: 206ms	remaining: 25.5s
8:	learn: 0.4741938	total: 209ms	remaining: 23s
9:	learn: 0.4728501	total: 213ms	remaining: 21s
10:	learn: 0.4719775	total: 215ms	remaining: 19.3s
11:	learn: 0.4706343	total: 219ms	remaining: 18s
12:	learn: 0.4698219	total: 222ms	remaining: 16.9s
13:	learn: 0.4687005	total: 226ms	remaining: 15.9s
14:	learn: 0.4678652	total: 229ms	remaining: 15.1s
15:	learn: 0.4671173	total: 233ms	remaining: 14.3s
16:	learn: 0.4662968	total: 236ms	remaining: 13.6s
17:	learn: 0.4657529	total: 240ms	remaining: 13.1s
18:	learn: 0.4648659	total: 244ms	remaining: 12.6s
19:	learn: 0.464178

167:	learn: 0.4447985	total: 829ms	remaining: 4.11s
168:	learn: 0.4447176	total: 833ms	remaining: 4.1s
169:	learn: 0.4446664	total: 837ms	remaining: 4.08s
170:	learn: 0.4446310	total: 841ms	remaining: 4.08s
171:	learn: 0.4445506	total: 844ms	remaining: 4.06s
172:	learn: 0.4444808	total: 847ms	remaining: 4.05s
173:	learn: 0.4444147	total: 851ms	remaining: 4.04s
174:	learn: 0.4443578	total: 856ms	remaining: 4.04s
175:	learn: 0.4442856	total: 864ms	remaining: 4.04s
176:	learn: 0.4442374	total: 869ms	remaining: 4.04s
177:	learn: 0.4441685	total: 875ms	remaining: 4.04s
178:	learn: 0.4441030	total: 880ms	remaining: 4.04s
179:	learn: 0.4440141	total: 887ms	remaining: 4.04s
180:	learn: 0.4439249	total: 891ms	remaining: 4.03s
181:	learn: 0.4438594	total: 911ms	remaining: 4.09s
182:	learn: 0.4437515	total: 933ms	remaining: 4.17s
183:	learn: 0.4437225	total: 937ms	remaining: 4.16s
184:	learn: 0.4436217	total: 942ms	remaining: 4.15s
185:	learn: 0.4435558	total: 951ms	remaining: 4.16s
186:	learn: 0

364:	learn: 0.4344461	total: 1.64s	remaining: 2.85s
365:	learn: 0.4343777	total: 1.65s	remaining: 2.85s
366:	learn: 0.4343461	total: 1.65s	remaining: 2.84s
367:	learn: 0.4343242	total: 1.65s	remaining: 2.84s
368:	learn: 0.4343219	total: 1.66s	remaining: 2.83s
369:	learn: 0.4342939	total: 1.66s	remaining: 2.82s
370:	learn: 0.4342393	total: 1.66s	remaining: 2.82s
371:	learn: 0.4341824	total: 1.67s	remaining: 2.81s
372:	learn: 0.4341380	total: 1.67s	remaining: 2.81s
373:	learn: 0.4340731	total: 1.67s	remaining: 2.8s
374:	learn: 0.4340573	total: 1.68s	remaining: 2.8s
375:	learn: 0.4340216	total: 1.68s	remaining: 2.79s
376:	learn: 0.4340122	total: 1.69s	remaining: 2.79s
377:	learn: 0.4339612	total: 1.69s	remaining: 2.78s
378:	learn: 0.4339529	total: 1.69s	remaining: 2.77s
379:	learn: 0.4338788	total: 1.7s	remaining: 2.77s
380:	learn: 0.4338323	total: 1.7s	remaining: 2.76s
381:	learn: 0.4337801	total: 1.71s	remaining: 2.76s
382:	learn: 0.4337261	total: 1.72s	remaining: 2.77s
383:	learn: 0.43

538:	learn: 0.4277735	total: 2.29s	remaining: 1.96s
539:	learn: 0.4277383	total: 2.29s	remaining: 1.95s
540:	learn: 0.4277294	total: 2.3s	remaining: 1.95s
541:	learn: 0.4276803	total: 2.3s	remaining: 1.94s
542:	learn: 0.4276655	total: 2.3s	remaining: 1.94s
543:	learn: 0.4276137	total: 2.31s	remaining: 1.93s
544:	learn: 0.4275457	total: 2.31s	remaining: 1.93s
545:	learn: 0.4275211	total: 2.31s	remaining: 1.92s
546:	learn: 0.4275201	total: 2.32s	remaining: 1.92s
547:	learn: 0.4274774	total: 2.32s	remaining: 1.91s
548:	learn: 0.4274729	total: 2.32s	remaining: 1.91s
549:	learn: 0.4274031	total: 2.33s	remaining: 1.9s
550:	learn: 0.4273874	total: 2.33s	remaining: 1.9s
551:	learn: 0.4273208	total: 2.34s	remaining: 1.9s
552:	learn: 0.4273196	total: 2.34s	remaining: 1.89s
553:	learn: 0.4272795	total: 2.35s	remaining: 1.89s
554:	learn: 0.4272457	total: 2.35s	remaining: 1.89s
555:	learn: 0.4272377	total: 2.36s	remaining: 1.88s
556:	learn: 0.4272329	total: 2.36s	remaining: 1.88s
557:	learn: 0.4272

728:	learn: 0.4219587	total: 3.09s	remaining: 1.15s
729:	learn: 0.4219006	total: 3.09s	remaining: 1.14s
730:	learn: 0.4218769	total: 3.1s	remaining: 1.14s
731:	learn: 0.4218500	total: 3.1s	remaining: 1.14s
732:	learn: 0.4218017	total: 3.1s	remaining: 1.13s
733:	learn: 0.4217972	total: 3.11s	remaining: 1.13s
734:	learn: 0.4217880	total: 3.11s	remaining: 1.12s
735:	learn: 0.4217413	total: 3.12s	remaining: 1.12s
736:	learn: 0.4217296	total: 3.12s	remaining: 1.11s
737:	learn: 0.4217267	total: 3.12s	remaining: 1.11s
738:	learn: 0.4217002	total: 3.13s	remaining: 1.1s
739:	learn: 0.4216801	total: 3.13s	remaining: 1.1s
740:	learn: 0.4216777	total: 3.13s	remaining: 1.09s
741:	learn: 0.4216568	total: 3.14s	remaining: 1.09s
742:	learn: 0.4216444	total: 3.14s	remaining: 1.08s
743:	learn: 0.4216007	total: 3.14s	remaining: 1.08s
744:	learn: 0.4215837	total: 3.15s	remaining: 1.08s
745:	learn: 0.4215107	total: 3.15s	remaining: 1.07s
746:	learn: 0.4215101	total: 3.15s	remaining: 1.07s
747:	learn: 0.421

888:	learn: 0.4179805	total: 3.73s	remaining: 466ms
889:	learn: 0.4179594	total: 3.74s	remaining: 462ms
890:	learn: 0.4179115	total: 3.74s	remaining: 458ms
891:	learn: 0.4178851	total: 3.75s	remaining: 454ms
892:	learn: 0.4178713	total: 3.75s	remaining: 450ms
893:	learn: 0.4178615	total: 3.77s	remaining: 447ms
894:	learn: 0.4177719	total: 3.77s	remaining: 442ms
895:	learn: 0.4177573	total: 3.77s	remaining: 438ms
896:	learn: 0.4177296	total: 3.78s	remaining: 434ms
897:	learn: 0.4176731	total: 3.78s	remaining: 430ms
898:	learn: 0.4176724	total: 3.78s	remaining: 425ms
899:	learn: 0.4176501	total: 3.79s	remaining: 421ms
900:	learn: 0.4175962	total: 3.79s	remaining: 417ms
901:	learn: 0.4175832	total: 3.79s	remaining: 412ms
902:	learn: 0.4175696	total: 3.8s	remaining: 408ms
903:	learn: 0.4175375	total: 3.8s	remaining: 404ms
904:	learn: 0.4175340	total: 3.81s	remaining: 400ms
905:	learn: 0.4175269	total: 3.81s	remaining: 395ms
906:	learn: 0.4175013	total: 3.81s	remaining: 391ms
907:	learn: 0.